<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from openvino.runtime import Core
import torch.nn as nn

class ModelComparator:
    def __init__(self, pytorch_model, openvino_model_path, device='CPU'):
        """
        PyTorchモデルとOpenVINOモデルの比較を行うクラス

        Args:
            pytorch_model: PyTorchモデル
            openvino_model_path: OpenVINOモデルのパス
            device: 実行デバイス（'CPU' or 'GPU'）
        """
        self.pytorch_model = pytorch_model
        self.pytorch_model.eval()

        # OpenVINOモデルの読み込み
        ie = Core()
        self.openvino_model = ie.read_model(openvino_model_path)
        self.compiled_model = ie.compile_model(self.openvino_model, device)

        # 中間層の出力を保存するためのフック
        self.pytorch_features = {}
        self.layer_names = []
        self._register_hooks()

    def _register_hooks(self):
        """PyTorchモデルの各層にフックを登録"""
        def get_activation(name):
            def hook(model, input, output):
                self.pytorch_features[name] = output.detach()
            return hook

        for name, module in self.pytorch_model.named_modules():
            if isinstance(module, (nn.Conv2d, nn.Linear, nn.ReLU)):
                self.layer_names.append(name)
                module.register_forward_hook(get_activation(name))

    def compare_outputs(self, input_tensor, rtol=1e-3, atol=1e-3):
        """
        両モデルの出力を比較

        Args:
            input_tensor: 入力テンソル
            rtol: 相対許容誤差
            atol: 絶対許容誤差

        Returns:
            比較結果の辞書
        """
        # PyTorchモデルの実行
        pytorch_output = self.pytorch_model(input_tensor)

        # OpenVINOモデルの実行のための入力準備
        input_numpy = input_tensor.numpy()
        openvino_output = self.compiled_model([input_numpy])[0]

        # 最終出力の比較
        final_output_match = np.allclose(
            pytorch_output.detach().numpy(),
            openvino_output,
            rtol=rtol,
            atol=atol
        )

        results = {
            'final_output_match': final_output_match,
            'layer_comparisons': {},
            'max_differences': {}
        }

        # 中間層の出力を取得して比較
        for output_tensor in self.compiled_model.outputs:
            layer_name = output_tensor.get_names()[0]
            if layer_name in self.pytorch_features:
                pytorch_feature = self.pytorch_features[layer_name].numpy()
                openvino_feature = output_tensor.get_tensor()

                # 出力の形状が一致するか確認
                if pytorch_feature.shape != openvino_feature.shape:
                    results['layer_comparisons'][layer_name] = False
                    continue

                # 出力値の比較
                is_close = np.allclose(
                    pytorch_feature,
                    openvino_feature,
                    rtol=rtol,
                    atol=atol
                )

                max_diff = np.max(np.abs(pytorch_feature - openvino_feature))
                results['layer_comparisons'][layer_name] = is_close
                results['max_differences'][layer_name] = float(max_diff)

        return results

    def print_comparison_results(self, results):
        """比較結果を整形して表示"""
        print("=== Model Comparison Results ===")
        print(f"Final Output Match: {results['final_output_match']}\n")

        print("Layer-wise Comparison Results:")
        for layer_name, matches in results['layer_comparisons'].items():
            max_diff = results['max_differences'].get(layer_name, float('nan'))
            status = "✓" if matches else "✗"
            print(f"{layer_name}: {status} (Max difference: {max_diff:.6f})")

# 使用例
def example_usage():
    # PyTorchモデルとOpenVINOモデルのパスを指定
    pytorch_model = torch.nn.Sequential(
        torch.nn.Conv2d(3, 64, 3),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2)
    )
    openvino_model_path = "path/to/model.xml"

    # 比較器の初期化
    comparator = ModelComparator(pytorch_model, openvino_model_path)

    # テスト入力の作成
    input_tensor = torch.randn(1, 3, 224, 224)

    # 比較の実行
    results = comparator.compare_outputs(input_tensor)

    # 結果の表示
    comparator.print_comparison_results(results)

In [ ]:
import torch
import numpy as np
from openvino.runtime import Core

# PyTorchモデルの例: 事前学習済みResNetモデル
import torchvision.models as models
pytorch_model = models.resnet18(pretrained=True)
pytorch_model.eval()  # 推論モードに設定

# OpenVINOモデルの読み込み
ie = Core()
openvino_model_path = 'resnet18.xml'  # OpenVINO IR形式のモデルファイル (.xml)
openvino_model = ie.read_model(model=openvino_model_path)
compiled_model = ie.compile_model(openvino_model, device_name="CPU")

# 各レイヤー出力を取得するためのユーティリティ関数
def get_pytorch_layer_outputs(model, input_tensor):
    outputs = {}
    hooks = []

    def hook_fn(module, input, output):
        outputs[module] = output.detach().cpu().numpy()

    # 各レイヤーにフックを追加
    for layer in model.modules():
        if len(list(layer.children())) == 0:  # 子モジュールがない場合にのみフックを追加
            hooks.append(layer.register_forward_hook(hook_fn))

    # 順伝播実行
    with torch.no_grad():
        model(input_tensor)

    # フックを削除
    for hook in hooks:
        hook.remove()

    return outputs

def get_openvino_layer_outputs(compiled_model, input_data):
    outputs = {}
    request = compiled_model.create_infer_request()

    # 各中間レイヤーの出力を取得するためのノードを指定
    for node in compiled_model.outputs:
        outputs[node.get_node().friendly_name] = request.output_tensor(node).data

    return outputs

# 同じランダム入力を生成
input_tensor = torch.rand(1, 3, 224, 224)  # 例としてバッチサイズ1、3チャネル、224x224サイズの入力
input_data = input_tensor.numpy()

# PyTorchモデルの出力を取得
pytorch_outputs = get_pytorch_layer_outputs(pytorch_model, input_tensor)

# OpenVINOモデルの出力を取得
openvino_outputs = get_openvino_layer_outputs(compiled_model, input_data)

# 出力の比較
for layer_name, pytorch_output in pytorch_outputs.items():
    openvino_output = openvino_outputs.get(layer_name)
    if openvino_output is None:
        print(f"{layer_name}の出力がOpenVINOモデルに存在しません")
        continue

    # ここではMean Squared Errorで出力を比較
    mse = np.mean((pytorch_output - openvino_output) ** 2)
    print(f"レイヤー {layer_name} の出力差 (MSE): {mse}")

In [ ]:
import torch
import numpy as np
from openvino.runtime import Core
import torch.nn as nn

class ModelComparator:
    def __init__(self, pytorch_model, openvino_model_path, device='CPU'):
        self.pytorch_model = pytorch_model
        self.pytorch_model.eval()

        # OpenVINOモデルのロード
        ie = Core()
        self.openvino_model = ie.read_model(openvino_model_path)
        self.compiled_model = ie.compile_model(self.openvino_model, device)

        # 活性化値を保存する辞書
        self.pytorch_activations = {}
        self.layer_names = []

        # PyTorchモデルの各レイヤーにhookを追加
        def get_activation(name):
            def hook(model, input, output):
                self.pytorch_activations[name] = output.detach().numpy()
            return hook

        # すべてのレイヤーにhookを登録
        for name, layer in pytorch_model.named_modules():
            if isinstance(layer, (nn.Conv2d, nn.Linear, nn.ReLU, nn.MaxPool2d)):
                self.layer_names.append(name)
                layer.register_forward_hook(get_activation(name))

    def compare_outputs(self, input_tensor, rtol=1e-3, atol=1e-3):
        """
        PyTorchとOpenVINOモデルの出力を比較

        引数:
            input_tensor: PyTorchテンソル形式の入力
            rtol: 相対許容誤差
            atol: 絶対許容誤差
        """
        # PyTorchモデルを実行
        with torch.no_grad():
            pytorch_output = self.pytorch_model(input_tensor)

        # OpenVINO入力形式に変換
        input_numpy = input_tensor.numpy()

        # OpenVINOモデルを実行
        openvino_output = self.compiled_model([input_numpy])[0]

        # モデル全体の出力を比較
        is_close = np.allclose(pytorch_output.numpy(), openvino_output, rtol=rtol, atol=atol)
        print(f"\nモデル全体の出力比較: {'一致' if is_close else '不一致'}")

        if not is_close:
            print(f"最大差異: {np.max(np.abs(pytorch_output.numpy() - openvino_output))}")
            print(f"平均差異: {np.mean(np.abs(pytorch_output.numpy() - openvino_output))}")

        # レイヤーごとの出力比較
        print("\nレイヤーごとの出力比較:")
        for name in self.layer_names:
            if name in self.pytorch_activations:
                pytorch_act = self.pytorch_activations[name]

                # OpenVINOモデルから対応するレイヤーの出力を取得
                try:
                    node_name = name.replace('.', '/')
                    openvino_act = self.compiled_model.get_tensor(node_name).data

                    is_close = np.allclose(pytorch_act, openvino_act, rtol=rtol, atol=atol)
                    print(f"\nレイヤー {name}:")
                    print(f"一致状態: {'一致' if is_close else '不一致'}")

                    if not is_close:
                        print(f"最大差異: {np.max(np.abs(pytorch_act - openvino_act))}")
                        print(f"平均差異: {np.mean(np.abs(pytorch_act - openvino_act))}")
                except Exception as e:
                    print(f"レイヤー {name}の比較中にエラーが発生: {str(e)}")

    def get_statistics(self, activations):
        """活性化値の統計情報を返す"""
        return {
            'mean': np.mean(activations),
            'std': np.std(activations),
            'min': np.min(activations),
            'max': np.max(activations)
        }

In [ ]:

import torch
import numpy as np
import openvino.inference_engine as ie

# PyTorchモデルのロード
torch_model = torch.load('pytorch_model.pth')
torch_model.eval()

# OpenVINOモデルのロード
core = ie.Core()
model = core.read_network('openvino_model.xml', 'openvino_model.bin')
exec_net = core.load_network(network=model, device_name='CPU')

# 入力データの準備
input_data = torch.randn(1, 3, 224, 224)
input_blob = next(iter(model.inputs))
net_input = {input_blob: input_data.numpy()}

# 指定したレイヤーの出力比較
target_layer = 'layer_name'  # 比較したいレイヤー名

# PyTorchの順伝播
with torch.no_grad():
    torch_output = torch_model(input_data)
    torch_output = torch_output[target_layer]

# OpenVINOの順伝播
infer_request = exec_net.infer(inputs=net_input)
openvino_output = infer_request[target_layer]

# 出力の比較 (例: RMSE)
error = np.sqrt(np.mean((torch_output.numpy() - openvino_output)**2))
print(f"RMSE: {error}")

# 可視化 (例: Matplotlib)
import matplotlib.pyplot as plt
plt.imshow(torch_output[0, 0].numpy())
plt.title('PyTorch Output')
plt.show()
plt.imshow(openvino_output[0, 0])
plt.title('OpenVINO Output')
plt.show()